In [1]:
from IPython.core.display import display, HTML, Javascript
from string import Template
import json, os
import pandas as pd
import numpy as np

Data files must be inside 'data' folder next to the jupyter files while trying to reproduce the code.
Data canbe found under Big-Data-Project\datasets\Data used for analysis -Yufan\data 

In [2]:
notebook_path = os.path.abspath("Shooting_VS_Income.ipynb")
historic_csv = os.path.join(os.path.dirname(notebook_path), "data/", "historic_incident_with_zipcode_cleaned.csv")
historic = pd.read_csv(historic_csv)
current_csv = os.path.join(os.path.dirname(notebook_path), "data/", "historic_incident_with_zipcode2020_cleaned.csv")
current = pd.read_csv(current_csv)
income_csv = os.path.join(os.path.dirname(notebook_path), "data/", "Median_Income_Byzip_Avg.csv")
income = pd.read_csv(income_csv)
current.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/yufan/Jupyter/Jupyter_D3_Lab/data/Median_Income_Byzip_Avg.csv'

In [ ]:
bigdata = historic.append(current, ignore_index=True)

In [ ]:
bigdata['YEAR'] = pd.DatetimeIndex(bigdata['OCCUR_DATE']).year
filtered_data = bigdata.loc[(bigdata['YEAR'] <= 2019) & (bigdata['YEAR'] >= 2017)]
grpby = filtered_data.groupby('ZipCode').size().reset_index(name='counts')
grpby.head()

In [ ]:
shooting_income =  pd.merge(income, grpby, on='ZipCode', how='left')
shooting_income['counts'] = shooting_income['counts'].fillna(0)
shooting_income['counts']=shooting_income['counts'].astype('int')
data = shooting_income.to_dict(orient='records')
#shooting_income.to_csv('shooting_vs_income.csv')
#print(data)

In [ ]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    console.log(d3);
    window.d3 = d3;
});

In [ ]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [ ]:
css_text = ''''''

In [ ]:
js_text_template = Template('''
  var margin = ({top: 20, bottom: 40, left: 50, right: 20})  ;
  var visWidth = 900 - margin.left - margin.right;
  var visHeight = 500 - margin.top - margin.bottom;
  
  var data = $python_data;
  
  var maxCount = d3.max(data, d => d.counts);
  var maxIncome = d3.max(data, d => d.Median_Household_Income);
  
  var countScale = d3.scaleLinear()
    .domain([0, maxCount])
    .range([visHeight, 0])
    .nice()
  
  var incomeScale = d3.scaleLinear()
    .domain([0, maxIncome])
    .range([0, visWidth])
    .nice()
  
  var xAxis = d3.axisBottom(incomeScale) 
     .tickFormat(d => d/1000 + 'k');
  
  var yAxis = d3.axisLeft(countScale);
  
  var svg = d3.select("#$graphdiv")
      .append('svg')
      .attr('width', visWidth + margin.left + margin.right)
      .attr('height', visHeight + margin.top + margin.bottom); 
  
  var g = svg.append("g")
      .attr('transform', `translate(${margin.left}, ${margin.top})`);  
  
  g.selectAll('circle')
    .data(data)
    .join('circle')
      .attr('cx', d => incomeScale(d.Median_Household_Income))
      .attr('cy', d => countScale(d.counts))
      .attr('r', 7)
      .attr('fill', 'cornflowerblue')  
  
  g.append('g')
      .call(yAxis);  
  
  g.append('g')
      .attr('transform', `translate(0, ${visHeight})`)
      .call(xAxis)  
  
  g.append('text')
      .attr('fill', 'black')
      .attr('font-family', 'sans-serif')
      .attr('x', visWidth / 8)
      .attr('y', 0)
      .style('font-size', 18)
      .text("Median Household Income vs Number of Gun Violence (2017-2019)");
  
  g.append('text')
      .attr('fill', 'black')
      .attr('font-family', 'sans-serif')
      .attr('x', visWidth / 3)
      .attr('y', 20)
      .style('font-size', 18)
      .text("by Postal Code Area");  
''')

In [ ]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')
js_text = js_text_template.safe_substitute(python_data=json.dumps(data),
                                       graphdiv='graph-div')
HTML(html_template.safe_substitute(css_text=css_text, js_text=js_text))